In [15]:
!pip3 install pandas geopandas shapely fiona geopy sqlalchemy geoalchemy2 psycopg2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [17]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point
import fiona
from sqlalchemy import create_engine
# import sqlalchemy
# import geoalchemy2
# import psycopg2
from geopy.geocoders import Nominatim
import requests
import re
import json

In [2]:
banksCA = pd.read_csv("./data/processed/OFFICES2_CALIF_geocodio.csv")
banksAZNVOR = pd.read_csv("./data/processed/OFFICES2_AZ_NV_OR_geocodio.csv")

/Users/work/.pyenv/versions/3.9.6/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (35,48,54,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
banksCA.columns.values.tolist()

['ADDRESS',
 'ADDRESS_FULL',
 'BKCLASS',
 'CBSA',
 'CBSA_DIV',
 'CBSA_DIV_FLG',
 'CBSA_DIV_NO',
 'CBSA_METRO',
 'CBSA_METRO_FLG',
 'CBSA_METRO_NAME',
 'CBSA_MICRO_FLG',
 'CBSA_NO',
 'CERT',
 'CITY',
 'COUNTY',
 'CSA',
 'CSA_FLG',
 'CSA_NO',
 'ESTYMD',
 'FI_UNINUM',
 'MAINOFF',
 'NAME',
 'OFFNAME',
 'OFFNUM',
 'RUNDATE',
 'SERVTYPE',
 'STALP',
 'STCNTY',
 'STNAME',
 'UNINUM',
 'ZIP',
 'Latitude',
 'Longitude',
 'Accuracy Score',
 'Accuracy Type',
 'Number',
 'Street',
 'Unit Type',
 'Unit Number',
 'City',
 'State',
 'County',
 'Zip',
 'Country',
 'Source',
 'ZIP+4 Range Start',
 'ZIP+4 Range End (or blank if no range)',
 'ZIP 9 Range Start',
 'ZIP 9 Range End (or blank if no range)',
 'Record Type Code',
 'Record Type Description',
 'Carrier Route ID',
 'Carrier Route Description',
 'Building or Firm Name',
 'Government Building Code',
 'Government Building Description',
 'Facility Code',
 'Facility Description',
 'City Delivery?',
 'Valid delivery area?',
 'Exact Match?',
 'Census Yea

In [4]:
banksAZNVOR.columns.values.tolist()

['ADDRESS',
 'ADDRESS_FULL',
 'BKCLASS',
 'CBSA',
 'CBSA_DIV',
 'CBSA_DIV_FLG',
 'CBSA_DIV_NO',
 'CBSA_METRO',
 'CBSA_METRO_FLG',
 'CBSA_METRO_NAME',
 'CBSA_MICRO_FLG',
 'CBSA_NO',
 'CERT',
 'CITY',
 'COUNTY',
 'CSA',
 'CSA_FLG',
 'CSA_NO',
 'ESTYMD',
 'FI_UNINUM',
 'MAINOFF',
 'NAME',
 'OFFNAME',
 'OFFNUM',
 'RUNDATE',
 'SERVTYPE',
 'STALP',
 'STCNTY',
 'STNAME',
 'UNINUM',
 'ZIP',
 'Latitude',
 'Longitude',
 'Accuracy Score',
 'Accuracy Type',
 'Number',
 'Street',
 'Unit Type',
 'Unit Number',
 'City',
 'State',
 'County',
 'Zip',
 'Country',
 'Source',
 'Census Year',
 'State FIPS',
 'County FIPS',
 'Place Name',
 'Place FIPS',
 'Census Tract Code',
 'Census Block Code',
 'Census Block Group',
 'Full FIPS (block)',
 'Full FIPS (tract)',
 'Metro/Micro Statistical Area Name',
 'Metro/Micro Statistical Area Code',
 'Metro/Micro Statistical Area Type',
 'Combined Statistical Area Name',
 'Combined Statistical Area Code',
 'Metropolitan Division Area Name',
 'Metropolitan Division Area 

In [3]:
fields = []
for field in banksCA.columns.values.tolist():
    if field not in banksAZNVOR.columns.values.tolist():
        fields.append(field)
print(fields)

['ZIP+4 Range Start', 'ZIP+4 Range End (or blank if no range)', 'ZIP 9 Range Start', 'ZIP 9 Range End (or blank if no range)', 'Record Type Code', 'Record Type Description', 'Carrier Route ID', 'Carrier Route Description', 'Building or Firm Name', 'Government Building Code', 'Government Building Description', 'Facility Code', 'Facility Description', 'City Delivery?', 'Valid delivery area?', 'Exact Match?']


In [4]:
banksCA.drop(fields, axis=1, inplace=True)
banksCA

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Some other race alone/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage"
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.000,0.0,12.0,0.000
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.000,0.0,12.0,0.000
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.493,10.0,15.0,0.137,10.0,15.0,1.000,0.0,12.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6327,6 Hutton Centre Drive Suite 100,"6 Hutton Centre Drive Suite 100, Santa Ana, CA...",NM,"Los Angeles-Long Beach-Anaheim, CA","Anaheim-Santa Ana-Irvine, CA",1,11244,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.226,84.0,65.0,0.032,36.0,35.0,0.429,48.0,43.0,0.571
6328,750 B Street Suite 3110,"750 B Street Suite 3110, San Diego, CA 92101",NM,"San Diego-Chula Vista-Carlsbad, CA",NaN,0,0,41740,1,"San Diego-Chula Vista-Carlsbad, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000
6329,"4675 Macarthur Court, Suite 1600","4675 Macarthur Court, Suite 1600, Newport Beac...",NM,"Los Angeles-Long Beach-Anaheim, CA","Anaheim-Santa Ana-Irvine, CA",1,11244,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.365,30.0,27.0,0.030,19.0,30.0,0.633,11.0,18.0,0.367
6330,"520 Newport Center Drive, Suite 700","520 Newport Center Drive, Suite 700, Newport B...",N,"Los Angeles-Long Beach-Anaheim, CA","Anaheim-Santa Ana-Irvine, CA",1,11244,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000


In [5]:
banksDF = pd.concat([banksCA, banksAZNVOR])

In [6]:
banksDF.reset_index(drop=True, inplace=True)
banksDF

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Some other race alone/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage"
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.493,10.0,15.0,0.137,10.0,15.0,1.0,0.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,5370 Kietzke Lane,"5370 Kietzke Lane, Reno, NV 89511",N,"Reno, NV",NaN,0,0,39900,1,"Reno, NV",...,0.234,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0
8695,2405 West Burnside Street,"2405 West Burnside Street, Portland, OR 97210",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.149,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
8696,947 Sw Broadway,"947 Sw Broadway, Portland, OR 97205",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
8697,"2360 Corporate Circle, Suite 400","2360 Corporate Circle, Suite 400, Henderson, N...",SB,"Las Vegas-Henderson-Paradise, NV",NaN,0,0,29820,1,"Las Vegas-Henderson-Paradise, NV",...,0.000,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0


In [7]:
banksDF['geometry'] = banksDF.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)


In [12]:
len(banksDF['geometry'])

8699

In [8]:
banksDF

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage",geometry
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.416419 34.058859)
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.416419 34.058859)
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.395607 37.788001)
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.395607 37.788001)
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,10.0,15.0,0.137,10.0,15.0,1.0,0.0,12.0,0.0,POINT (-122.399668 37.793118)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,5370 Kietzke Lane,"5370 Kietzke Lane, Reno, NV 89511",N,"Reno, NV",NaN,0,0,39900,1,"Reno, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-119.790735 39.469506)
8695,2405 West Burnside Street,"2405 West Burnside Street, Portland, OR 97210",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.700313 45.524249)
8696,947 Sw Broadway,"947 Sw Broadway, Portland, OR 97205",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.680924 45.517641)
8697,"2360 Corporate Circle, Suite 400","2360 Corporate Circle, Suite 400, Henderson, N...",SB,"Las Vegas-Henderson-Paradise, NV",NaN,0,0,29820,1,"Las Vegas-Henderson-Paradise, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-115.088583 36.025051)


In [9]:
banksGDF = gpd.GeoDataFrame(banksDF, crs="EPSG:4326", geometry=banksDF.geometry)

In [10]:
banksGDF

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage",geometry
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.41642 34.05886)
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.41642 34.05886)
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.39561 37.78800)
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.39561 37.78800)
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,10.0,15.0,0.137,10.0,15.0,1.0,0.0,12.0,0.0,POINT (-122.39967 37.79312)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,5370 Kietzke Lane,"5370 Kietzke Lane, Reno, NV 89511",N,"Reno, NV",NaN,0,0,39900,1,"Reno, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-119.79073 39.46951)
8695,2405 West Burnside Street,"2405 West Burnside Street, Portland, OR 97210",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.70031 45.52425)
8696,947 Sw Broadway,"947 Sw Broadway, Portland, OR 97205",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.68092 45.51764)
8697,"2360 Corporate Circle, Suite 400","2360 Corporate Circle, Suite 400, Henderson, N...",SB,"Las Vegas-Henderson-Paradise, NV",NaN,0,0,29820,1,"Las Vegas-Henderson-Paradise, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-115.08858 36.02505)


In [16]:
banksGDF.to_file("data/processed/OFFICES2_CA_AZ_NV_OR_geocodio.geojson", driver="GeoJSON")

In [13]:
banksGDF.geometry.head()

0    POINT (-118.41642 34.05886)
1    POINT (-118.41642 34.05886)
2    POINT (-122.39561 37.78800)
3    POINT (-122.39561 37.78800)
4    POINT (-122.39967 37.79312)
Name: geometry, dtype: geometry

In [3]:
# banks = gpd.read_file('./data/processed/OFFICES2_CA_AZ_NV_OR_geocodio.geojson')
b = gpd.read_file('../sql/data/processed/pts_fdic_ncua_in_ua.csv')
b['pts'] = gpd.GeoSeries.from_wkt(b['pts'])

In [4]:
# type(banks.geometry.head(1))
# type(b.pts.head(1))
b = gpd.GeoDataFrame(b, geometry='pts')
b.head()

,id,ua,ua_id,pts,geometry
0,245045,"Riverdale, CA",75270,POINT (-119.86116 36.43119),None
1,015686,"Visalia, CA",90946,POINT (-119.29536 36.33079),None
2,056303,"Visalia, CA",90946,POINT (-119.29928 36.32968),None
3,146824340,"Visalia, CA",90946,POINT (-119.34658 36.21087),None
4,146824353,"Visalia, CA",90946,POINT (-119.34633 36.31263),None


In [15]:
body = {"locations":[[8.681495,49.41461],[8.686507,49.41943]],"range":[300,200]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': 'your-api-key',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('http://localhost:8080/ors/v2/isochrones/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

500 
{"error":{"code":3099,"message":"Unable to build an isochrone map."},"info":{"engine":{"version":"6.6.1","build_date":"2021-11-19T16:07:05Z"},"timestamp":1637344413632}}


In [5]:
coords = []

for i in b.pts:
    # print(re.sub("POINT", "", i))
    # print(type(i.x))
    xy = [i.x,i.y]
    coords.append(xy)
print(coords)

[[-119.861162, 36.431194], [-119.295364, 36.330791], [-119.299283, 36.329676], [-119.346582, 36.210871], [-119.346328, 36.31263], [-119.31336, 36.298137], [-119.331556, 36.358913], [-119.326593, 36.225977], [-119.277507, 36.325575], [-119.313399, 36.225718], [-119.296938, 36.3567], [-119.297944, 36.330395], [-119.345765, 36.209326], [-119.296326, 36.330423], [-119.314252, 36.296973], [-119.290655, 36.33063], [-119.313457, 36.309191], [-119.344432, 36.20806], [-119.141794, 36.295732], [-119.344965, 36.207943], [-119.296165, 36.3315], [-119.315879, 36.196524], [-119.351106, 36.328483], [-119.154408, 36.297656], [-119.314237, 36.307947], [-119.291464, 36.330436], [-119.345659, 36.208318], [-119.300254, 36.330386], [-119.34395, 36.208147], [-119.314247, 36.309069], [-119.331261, 36.225006], [-119.314195, 36.3061], [-119.336494, 36.225543], [-119.318851, 36.225417], [-119.292134, 36.329891], [-119.139397, 36.296679], [-119.348144, 36.341152], [-119.296829, 36.329677], [-119.325049, 36.22654

In [7]:
isos = []

for coord in coords:
    # print(coord)
    body = {
        'locations': [coord],
        'range': [1800]
        }

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': 'your-api-key',
        'Content-Type': 'application/json; charset=utf-8'
    }

    call = requests.post('http://localhost:8080/ors/v2/isochrones/driving-car', json=body, headers=headers)

    print(call.status_code, call.reason)
    # print(call.text)
    isos.append(call.text)

file=open('isos.txt','w')
for i in isos:
    file.writelines([i])
file.close()

200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 
200 


In [47]:
feats = []
t = []
f = []

for iso in isos:
    # print(type(iso))
    # print(iso)
    str = json.dumps(iso)
    dict = json.loads(iso)
    # print(type(dict))
    # print(dict)
    if 'features' in dict:
        feats.append(dict['features'][0])
    else:
        f.append(dict)

print(len(feats))
print(len(f))
        
gdf = gpd.GeoDataFrame.from_features(feats)

engine = create_engine("postgresql://postgres:@localhost:5432/ca-banks")  

gdf.to_postgis("iso_30_min_drive_pts_fdic_ncua_in_ua", engine)

# study_area = json.loads("""
#  {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[36.394272, -18.626726], [36.394272, -18.558391], [36.489716, -18.558391], [36.489716, -18.626726], [36.394272, -18.626726]]]}}]}
# """)
# gdf = gpd.GeoDataFrame.from_features(study_area["features"])
# print(gdf.head())

7521
1


In [19]:
engine = create_engine("postgresql://postgres:@localhost:5432/ca-banks")  
gdf.to_postgis("iso_30_min_drive_pts_fdic_ncua_in_ua", engine)

In [35]:
centroidsDF = pd.read_csv("./data/processed/distance-to-banks-demographics-reduced.csv")
centroidsDF['geometry'] = centroidsDF.apply(lambda row: Point(row.longitude, row.latitude), axis=1)
centroidsGDF = gpd.GeoDataFrame(centroidsDF, crs="EPSG:4326", geometry=centroidsDF.geometry)


/usr/local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [36]:
centroidsGDFdrop = centroidsGDF.drop(['X', 'Y', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_black_or_african_american_american_indian_and_alaska_native', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_black_or_african_american_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_black_or_african_american_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_american_indian_and_alaska_native_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_american_indian_and_alaska_native_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_white_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_black_or_african_american_american_indian_and_alaska_native_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_black_or_african_american_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_american_indian_and_alaska_native_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_three_races_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_black_or_african_american_american_indian_and_alaska_native_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_black_or_african_american_american_indian_and_alaska_native_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_black_or_african_american_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_black_or_african_american_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_american_indian_and_alaska_native_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_white_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_other_', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_four_races_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races_white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races_white_black_or_african_american_american_indian_and_alaska_native_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races_white_black_or_african_american_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander_some_', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races_white_black_or_african_american_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races_white_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_five_races_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_some_', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_six_races', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_six_races_white_black_or_african_american_american_indian_and_alaska_native_asian_native_hawaiian_and_other_pacific_islander_', 'HubName'],  axis=1)
centroidsGDFdrop.rename(columns = {'total':'total', 'total_hispanic_or_latino':'total_hispanic_or_latino', 'total_not_hispanic_or_latino':'total_not_hispanic_or_latino', 'total_not_hispanic_or_latino_population_of_one_race':'one_race', 'total_not_hispanic_or_latino_population_of_one_race_white_alone':'one_race_white_alone', 'total_not_hispanic_or_latino_population_of_one_race_black_or_african_american_alone':'one_race_black_or_african_american_alone', 'total_not_hispanic_or_latino_population_of_one_race_american_indian_and_alaska_native_alone':'one_race_american_indian_and_alaska_native_alone', 'total_not_hispanic_or_latino_population_of_one_race_asian_alone':'one_race_asian_alone', 'total_not_hispanic_or_latino_population_of_one_race_native_hawaiian_and_other_pacific_islander_alone':'one_race_native_hawaiian_and_other_pacific_islander_alone', 'total_not_hispanic_or_latino_population_of_one_race_some_other_race_alone':'one_race_some_other_race_alone', 'total_not_hispanic_or_latino_population_of_two_or_more_races':'two_or_more_races', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races':'two_races', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_white_black_or_african_american':'two_races_white_black_or_african_american', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_white_american_indian_and_alaska_native':'two_races_white_american_indian_and_alaska_native', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_white_asian':'two_races_white_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_white_native_hawaiian_and_other_pacific_islander':'two_races_white_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_white_some_other_race':'two_races_white_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_black_or_african_american_american_indian_and_alaska_native':'two_races_black_or_african_american_american_indian_and_alaska_native', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_black_or_african_american_asian':'two_races_black_or_african_american_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander':'two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_black_or_african_american_some_other_race':'two_races_black_or_african_american_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_american_indian_and_alaska_native_asian':'two_races_american_indian_and_alaska_native_asian', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander':'two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_american_indian_and_alaska_native_some_other_race':'two_races_american_indian_and_alaska_native_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_asian_native_hawaiian_and_other_pacific_islander':'two_races_asian_native_hawaiian_and_other_pacific_islander', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_asian_some_other_race':'two_races_asian_some_other_race', 'total_not_hispanic_or_latino_population_of_two_or_more_races_population_of_two_races_native_hawaiian_and_other_pacific_islander_some_other_race':'two_races_native_hawaiian_and_other_pacific_islander_some_other_race'}, inplace = True)
centroidsGDFdrop['hubdist'] = centroidsGDFdrop['HubDist'].astype(float).round(1)
centroidsGDFdrop.columns.values.tolist()

['longitude',
 'latitude',
 'geographic_area_name',
 'total',
 'total_hispanic_or_latino',
 'total_not_hispanic_or_latino',
 'one_race',
 'one_race_white_alone',
 'one_race_black_or_african_american_alone',
 'one_race_american_indian_and_alaska_native_alone',
 'one_race_asian_alone',
 'one_race_native_hawaiian_and_other_pacific_islander_alone',
 'one_race_some_other_race_alone',
 'two_or_more_races',
 'two_races',
 'two_races_white_black_or_african_american',
 'two_races_white_american_indian_and_alaska_native',
 'two_races_white_asian',
 'two_races_white_native_hawaiian_and_other_pacific_islander',
 'two_races_white_some_other_race',
 'two_races_black_or_african_american_american_indian_and_alaska_native',
 'two_races_black_or_african_american_asian',
 'two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander',
 'two_races_black_or_african_american_some_other_race',
 'two_races_american_indian_and_alaska_native_asian',
 'two_races_american_indian_and_alaska_nativ

In [37]:
# centroidsGDFdrop.head()

def f(row):
    latinx = row['total_hispanic_or_latino']
    white = row['one_race_white_alone']+row['two_races_white_black_or_african_american']+row['two_races_white_american_indian_and_alaska_native']+row['two_races_white_asian']+row['two_races_white_native_hawaiian_and_other_pacific_islander']+row['two_races_white_some_other_race']
    black = row['one_race_black_or_african_american_alone']+row['two_races_white_black_or_african_american']+row['two_races_black_or_african_american_american_indian_and_alaska_native']+row['two_races_black_or_african_american_asian']+row['two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander']+row['two_races_black_or_african_american_some_other_race']
    native_cont = row['one_race_american_indian_and_alaska_native_alone']+row['two_races_black_or_african_american_american_indian_and_alaska_native']+row['two_races_white_american_indian_and_alaska_native']+row['two_races_american_indian_and_alaska_native_asian']+row['two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander']+row['two_races_american_indian_and_alaska_native_some_other_race']
    asian = row['one_race_asian_alone']+row['two_races_white_asian']+row['two_races_black_or_african_american_asian']+row['two_races_american_indian_and_alaska_native_asian']+row['two_races_asian_native_hawaiian_and_other_pacific_islander']+row['two_races_asian_some_other_race']
    native_isl = row['one_race_native_hawaiian_and_other_pacific_islander_alone']+row['two_races_white_native_hawaiian_and_other_pacific_islander']+row['two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander']+row['two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander']+row['two_races_asian_native_hawaiian_and_other_pacific_islander']+row['two_races_native_hawaiian_and_other_pacific_islander_some_other_race']
    other = row['one_race_some_other_race_alone']+row['two_races_white_some_other_race']+row['two_races_black_or_african_american_some_other_race']+row['two_races_american_indian_and_alaska_native_some_other_race']+row['two_races_asian_some_other_race']+row['two_races_native_hawaiian_and_other_pacific_islander_some_other_race']

    if latinx > white and latinx > black and latinx > native_cont and latinx > asian and latinx > native_isl and latinx > other:
        val = 'Latinx'
    elif white > latinx and white > black and white > native_cont and white > asian and white > native_isl and white > other:
        val = 'White'
    elif black > latinx and black > white and black > native_cont and black > asian and black > native_isl and black > other:
        val = 'Black'
    elif native_cont > latinx and native_cont > black and native_cont > white and native_cont > asian and native_cont > native_isl and native_cont > other:
        val = 'Native Continental'
    elif asian > latinx and asian > black and asian > native_cont and asian > white and asian > native_isl and asian > other:
        val = 'Asian'
    elif native_isl > latinx and native_isl > black and native_isl > native_cont and native_isl > asian and native_isl > white and native_isl > other:
        val = 'Native Islands'
    else:
        val = 'Other'
    return val

#create new column 'Good' using the function above
centroidsGDFdrop['block_group'] = centroidsGDFdrop.apply(f, axis=1)

#view DataFrame 
centroidsGDFdrop.head()

,longitude,latitude,geographic_area_name,total,total_hispanic_or_latino,total_not_hispanic_or_latino,one_race,one_race_white_alone,one_race_black_or_african_american_alone,one_race_american_indian_and_alaska_native_alone,...,two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander,two_races_american_indian_and_alaska_native_some_other_race,two_races_asian_native_hawaiian_and_other_pacific_islander,two_races_asian_some_other_race,two_races_native_hawaiian_and_other_pacific_islander_some_other_race,HubDist,pct_white,geometry,hubdist,block_group
0,-119.623943,38.714908,"Block 1109, Block Group 1, Census Tract 100, A...",1,0,1,1,1,0,0,...,0,0,0,0,0,16.871384,100,POINT (-119.62394 38.71491),16.9,White
1,-117.214601,33.806614,"Block 4008, Block Group 4, Census Tract 426.28...",103,88,15,15,3,8,0,...,0,0,0,0,0,1.164600,3,POINT (-117.21460 33.80661),1.2,Latinx
2,-117.242900,33.691122,"Block 3007, Block Group 3, Census Tract 427.17...",45,28,17,14,6,4,0,...,0,0,0,0,0,1.463574,13,POINT (-117.24290 33.69112),1.5,Latinx
3,-117.199569,33.695542,"Block 3025, Block Group 3, Census Tract 427.11...",42,14,28,27,21,3,0,...,0,0,0,0,0,1.130265,50,POINT (-117.19957 33.69554),1.1,White
4,-116.961900,33.749865,"Block 1023, Block Group 1, Census Tract 434.01...",21,7,14,10,4,5,1,...,0,0,0,0,0,0.168449,19,POINT (-116.96190 33.74986),0.2,Latinx


In [38]:
output = centroidsGDFdrop.drop(['HubDist','longitude','latitude','total_hispanic_or_latino','total_not_hispanic_or_latino','one_race','one_race_white_alone','one_race_black_or_african_american_alone','one_race_american_indian_and_alaska_native_alone','one_race_asian_alone','one_race_native_hawaiian_and_other_pacific_islander_alone','one_race_some_other_race_alone','two_or_more_races','two_races','two_races_white_black_or_african_american','two_races_white_american_indian_and_alaska_native','two_races_white_asian','two_races_white_native_hawaiian_and_other_pacific_islander','two_races_white_some_other_race','two_races_black_or_african_american_american_indian_and_alaska_native','two_races_black_or_african_american_asian','two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander','two_races_black_or_african_american_some_other_race','two_races_american_indian_and_alaska_native_asian','two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander','two_races_american_indian_and_alaska_native_some_other_race','two_races_asian_native_hawaiian_and_other_pacific_islander','two_races_asian_some_other_race','two_races_native_hawaiian_and_other_pacific_islander_some_other_race'], axis=1)
output

,geographic_area_name,total,pct_white,geometry,hubdist,block_group
0,"Block 1109, Block Group 1, Census Tract 100, A...",1,100,POINT (-119.62394 38.71491),16.9,White
1,"Block 4008, Block Group 4, Census Tract 426.28...",103,3,POINT (-117.21460 33.80661),1.2,Latinx
2,"Block 3007, Block Group 3, Census Tract 427.17...",45,13,POINT (-117.24290 33.69112),1.5,Latinx
3,"Block 3025, Block Group 3, Census Tract 427.11...",42,50,POINT (-117.19957 33.69554),1.1,White
4,"Block 1023, Block Group 1, Census Tract 434.01...",21,19,POINT (-116.96190 33.74986),0.2,Latinx
...,...,...,...,...,...,...
377586,"Block 1003, Block Group 1, Census Tract 6205.2...",225,52,POINT (-118.37845 33.87425),0.2,White
377587,"Block 1000, Block Group 1, Census Tract 5702.0...",115,4,POINT (-118.15184 33.87381),1.1,Latinx
377588,"Block 1009, Block Group 1, Census Tract 6031.0...",30,3,POINT (-118.29176 33.88045),0.5,Latinx
377589,"Block 1014, Block Group 1, Census Tract 5737, ...",65,66,POINT (-118.10612 33.82397),0.9,White


In [24]:
list = centroidsGDFdrop.columns.to_list()
for i in list:
    print(i)

longitude
latitude
geographic_area_name
total
total_hispanic_or_latino
total_not_hispanic_or_latino
one_race
one_race_white_alone
one_race_black_or_african_american_alone
one_race_american_indian_and_alaska_native_alone
one_race_asian_alone
one_race_native_hawaiian_and_other_pacific_islander_alone
one_race_some_other_race_alone
two_or_more_races
two_races
two_races_white_black_or_african_american
two_races_white_american_indian_and_alaska_native
two_races_white_asian
two_races_white_native_hawaiian_and_other_pacific_islander
two_races_white_some_other_race
two_races_black_or_african_american_american_indian_and_alaska_native
two_races_black_or_african_american_asian
two_races_black_or_african_american_native_hawaiian_and_other_pacific_islander
two_races_black_or_african_american_some_other_race
two_races_american_indian_and_alaska_native_asian
two_races_american_indian_and_alaska_native_native_hawaiian_and_other_pacific_islander
two_races_american_indian_and_alaska_native_some_other_ra

In [39]:
output.to_file("data/processed/distance-to-banks-block-demographics.geojson", driver="GeoJSON")